My solution to https://adventofcode.com/2023/day/14.

Rolling rocks and calculating weight on the north (Part 1) or east (Part 2) fence.

<ins>Part 1</ins> is solved using the functions below. 
- `get_perpendicular_mtx` first transposes the matrix so that every column is a string
- `calc_weight` calculates the weight of all rolling rocks 'O' in that column, taking into account that they can't go past square rocks '#'. This is done by splitting the column string into segments between '#' and taking note of the starting position of every new segment (the list ind_new). 

<ins>Part 2</ins> asks for the weight of the matrix on a fence after 1000000000 spin cycles (each with four tilts North, West, South, East). 
I did not even attempt to run my functions this many times! Instead, the solution below calculates the periodicity of the spin cycles (the matrix patterns repeat every 70 cycles) and the first point at which this periodicity occurs (at 80). The rest is maths. 


-Annette

In [17]:
#get file and info.

filename = 'day14_input.txt'

with open (filename,'r') as f:
    lines = f.read().splitlines()

len(lines)

100

In [18]:
#Part 1 Function

def get_perpendicular_mtx(lines: list) -> (str, int):
    '''
    Args:
        mtx : list of rows or lines from text
    Returns:
        vtx : list of columns from text up to first #
        distance : distance of top row from bottom row = len of lines
    '''
    mtx = lines.copy()
    lastcol = len(mtx[0])
    vtx = [''.join([line[i] for line in mtx]) for i in range(lastcol)]
    return vtx, len(lines)

def calc_weight(column: str, distance: int) -> int:
    '''
    Args:
        column : string representing vertical column from north fence to south fence
        distance : integer representing distance from north to south fence
    Returns:
        weight_on_fence : integer representing weight of all round rocks on fence
    '''
    #indices of spot after '#' that first 'O' will occupy after tilting
    inds_new = [i for i in range(1,distance+1,1) if column[i-1] == '#'] 
    inds_new.insert(0,0)

    #split column into segments that are separated by '#'
    segments = column.split('#')
 
    weight_on_fence = 0 #total
 
    #calculate weight of every 'O' and add it to the total
    for ind,segment in enumerate(segments):
        round_count = segment.count('O')
        weight_on_fence += sum([distance - inds_new[ind] - j for j in range(round_count)])
    return weight_on_fence


In [19]:
# Part 1 Solution
vtx, distance = get_perpendicular_mtx(lines)
total_weight = sum([calc_weight(col, distance) for col in vtx])
(f'Part 1 Answer: {total_weight}')

'Part 1 Answer: 109466'

In [29]:
#Part 2 Functions

def roll_rocks(column: str, direction: str) -> str:
    '''
    Args:
        column : string representing column, after 90 degree turn
        direction : one of 'N','S','E','W'
    Returns:
        tilted_column : new string where round rocks have rolled to the edge or to the next square rock
    '''
    # print(column, direction)

    segments = column.split('#')
    rolled_segments = []
    for segment in segments:
        round_count = segment.count('O')
        if direction in ['N','W']:
            rolled_segments.append('O' * round_count + '.' * (len(segment)-round_count))
        else:
            rolled_segments.append( '.' * (len(segment)-round_count) + 'O' * round_count)
    tilted_column = '#'.join(rolled_segments)
    return tilted_column


def tilt_mtx(mtx: list, direction: str):
    '''
    Args:
        mtx : list of strings
        direction : string, representing direction of tilt
    Other Functions Used:
        roll_rocks
    Returns:
        ttx : list of strings, tilted matrix
    '''
    vtx, distance = get_perpendicular_mtx(mtx)
    # print(vtx)
    newtx = [roll_rocks(column, direction) for column in vtx ]
    return newtx

def spin_cycle(mtx: list) -> (list, distance):
    '''
    Args:
        mtx : list of strings
    Other Functions Used:
        tilt_mtx
    Returns:
        mtx : updated list of strings after 1 spin cycle
        distance : len of the mtx list
    '''
    directions = list('NWSE')  
    for d in directions:
        mtx = tilt_mtx(mtx, d)
        # print(d, mtx)
    distance = len(mtx)
    return mtx, distance

def get_periodicity(mtx: list):
    '''
    Args:
        mtx : list of strings
    Other Functions Used:
        spin_cycle
    Returns:
        periodicity: int, number of cycles matrix patterns are repeated 
        start_period : int, 1st cycle in periodicity
    '''
    mtx = lines.copy()
    mtx_dict = {} #capture cycle numbers of the same matrices reoccuring
    for cycle in range(500):
        mtx,distance = spin_cycle(mtx)
        mtx_str = str(mtx)
        mt_occurs_at = mtx_dict.get(mtx_str,[])
        mt_occurs_at.append(cycle)
        mtx_dict[mtx_str] = mt_occurs_at
    first_repeating = [repeating for repeating in mtx_dict.values() if len(repeating)>2][0]
    diffs = [first_repeating[i+1]-first_repeating[i] for i in range(len(first_repeating)-1)]
    if len(set(diffs)) == 1: #diffs between cycles are the same
        periodicity = diffs[0]
        start_cycle = first_repeating[0]
    return periodicity, start_cycle

def get_mtx_at_1000000000(lines: list, periodicity: int, start_cycle: int) -> list:
    mtx = lines.copy()
    end_cycle = 1000000000
    periodicity, start_cycle = get_periodicity(mtx)
    cycle = (end_cycle) % periodicity + (start_cycle//periodicity + 1)*periodicity
    print(f'{end_cycle}th cycle is the same as cycle number {cycle}')
    for c in range(cycle):
        mtx, distance = spin_cycle(mtx)
    return mtx

    
def calc_weight_p2(mtx: list, direction = 'E'):
    '''
    Args:
        mtx : list of strings
        direction : string, representing direction of tilt
            : east by default, spin cycle ends east
    Returns:
        mtx_weight_on_fence : integer, total weight on fence
    '''
    distance = len(mtx[0])
    # print(distance, mtx)
    if direction in ['S','E']:
        total_weight = sum([sum([i for i in range(distance) if column[i] == 'O']) for column in mtx])
    else:
        total_weight = sum([sum([distance-i for i in range(distance) if column[i] == 'O']) for column in mtx])
    return total_weight

In [30]:
# Part 2 Solution
periodicity, start_cycle = get_periodicity(lines)
print(periodicity, start_cycle,'<<< periodicity from cycle')
end_mtx = get_mtx_at_1000000000(lines, periodicity, start_cycle)

vtx, distance = get_perpendicular_mtx(end_mtx)
end_weight = calc_weight_p2(vtx,'N')
print(f'Part 2 Answer: {end_weight}')



70 80 <<< periodicity from cycle
1000000000th cycle is the same as cycle number 160
Part 2 Answer: 94585
